# Equation of State Class

Initialize an EOS Object to use multiple times.
You can specify an equation of state and if the name is regocnized, the EOS will already have all of it's properties built in.  If not, you can specify the properties.

Here are the equations of state implemented in the EOS class. 


## 1. 'Standard'

The standard EOS is the default initialization. 

Constants
- rho_s = 1665.3 (central density (density at r = 0))
- h =  0.1  (smoothing length)
- k = 0.1 (equation of state constant)
- n = 1 (polytropic index)

### Density from pressure
- n = $(\frac{rho_sp}{363.44})^{(1/2.54)}$
- density = $\frac{236 n^{2.54} + n* m_{neutron}}{rho_s }$


### Pressure dependence on radius 

Classical case: 
- $ \frac{dP}{dr} = -m*$rho_NS($p$)$/(r^2 + 1e-20)$

Relativistic case:
- $rh = $rho_NS($p$)                            
- $\frac{dP}{dr} = -(p+rh)\frac{(p r^3 + m)}{(r^2 - 2 m r + 1e-20)}$



##### Tolman-Oppenheimer-Volkov equation
- constrains the structure of a spherically symmetric body of isotropic material which is in static gravitational equilibrium, as modelled by general relativity.
- When supplemented with an equation of state,  $F ( ρ , P ) = 0 $ which relates density to pressure, the Tolman–Oppenheimer–Volkoff equation completely determines the structure of a spherically symmetric body of isotropic material in equilibrium. 
- If terms of order  $1/c^{2}$ are neglected, the Tolman–Oppenheimer–Volkoff equation becomes the Newtonian hydrostatic equation, used to find the equilibrium structure of a spherically symmetric body of isotropic material when general-relativistic corrections are not important.

$ \frac{dP}{dr} = \frac{Gm}{r^2}  \rho (1 + \frac{P}{\rho c^2}) (1 + \frac{4 \pi r^3P}{mc^2})(1 + \frac{2 Gm}{rc^2})^{-1}$



### Mass dependence on radius 

- $\frac{dm}{dr} = $rho_NS($p$)$r^2$   




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py

In [10]:
rho_s = 1665.3 #  Central density (density at r = 0)
hc = 197.327           # Conversion factor in MeV fm (hut * c)
G = hc * 6.67259e-45   # Gravitational constant
Ms = 1.1157467e60      # Mass of the sun in kg
mn = 938.926           # Mass of neutron in MeV c^-2

def rho_NS(p): 
    """ density from pressure NS, source: https://github.com/zaman13/Modeling-of-Neutron-Stars/ """
    n = (p*rho_s/363.44)**(1./2.54)
    return (236. * n**2.54 + n *mn)/rho_s 

def dp_dr_NS(r,m,p, Classical = False): 
    """ pressure dependence on radius NS, source: https://github.com/zaman13/Modeling-of-Neutron-Stars/ """
    if Classical:                              # classical model
        y = -m*rho_NS(p)/(r**2 + 1e-20)
    else:                                      # relativistic model
        rh = rho_NS(p)                            
        y = -(p+rh)*(p*r**3 + m)/(r**2 - 2*m*r + 1e-20)
    return y

def dm_dr_NS(r,m,p):
    """ Mass dependence on radius (dm/dr)."""
    return rho_NS(p)*r**2

def initial_n(eos): # # Function for determining initial value of n(r=0)
    n = 1
    err = 1
    tol = 1e-15
    count = 0
    rho_s = eos.rho_s
    # Newton-Raphson method
    while err > tol : 
        count += 1
        fn = n*mn + 236*n**(2.54) - rho_s
        dfn = mn + 236*2.54*n**(1.54)
        temp = n - fn/dfn
        err = np.abs(n-temp)
        n = temp
    print("Newton-Raphson Converged after ", count, "iterations")
    return n

class EOS: 
    def __init__(self, name = 'standard', P= None, dp_dr= None, notes = "None"):
        """ Initialize an EOS Object to use multiple times.
        If the name is regocnized, the EOS will already have all of it's properties built in.  If not, you can specify the properties. 
        """
        self.name = name.lower()
        if self.name == 'standard':
            
            rho_s = 1665.3 #  Central density (density at r = 0)
            self.rho_s = rho_s
            self.M0 = (4*3.14159265*(G**3)*rho_s)**(-0.5)
            self.R0 =  G*self.M0
            self.rho = lambda x : rho_NS(x)
            self.dp_dr = lambda r,m,p, flag: dp_dr_NS(r,m,p, flag)
            self.dm_dr = lambda r, m, p: dm_dr_NS(r,m,p)
            self.notes = 'Standard EOS.'
            h = 0.1 # smoothing length
            k = 0.1 # equation of state constant
            n = 1 # polytropic index
            self.constants = {"h":(h, "smoothing length (h)"),"k": (k,"equation of state constant (k)"),"n" :(n,'polytropic index (n)'), 'rho_s': (rho_s, "Central density")}
            self.lmbda = lambda M,R : 2*k*(1+n)*np.pi**(-3/(2*n)) * (M*gamma(5/2+n)/R**3/gamma(1+n))**(1/n) / R**2  #is external force constant, ~ 2.01
        
        else:
            self.P = P
            self.dp_dr = dp_dr
            self.notes = notes
            
    def __str__(self):
        return "EOS Object. Type: " + self.name + "\nNotes: " + self.notes + "\n-------\n"+ str(self.constants) +  "\n\n For more information, see the imported EOS file."
    
    def P_eos(self, rho): 
        func = self.P
        return func(rho)